In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta
plt.style.use('seaborn-poster') 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
mars_weather_df = pd.read_csv("../local_datasets/mars_weather_data_cleaned.csv")

In [ ]:
mars_weather_df.columns

In [ ]:
mars_weather_df = mars_weather_df[
    ["terrestrial_date", "sol", "min_temp", "max_temp", "pressure", "min_gts_temp", "max_gts_temp"]
]

In [ ]:
print(mars_weather_df.shape)
mars_weather_df.head()

In [ ]:
mars_weather_df.isnull().sum(axis=0)

In [ ]:
mars_weather_df = mars_weather_df[
    (mars_weather_df.terrestrial_date > "2013-06-01") & (mars_weather_df.terrestrial_date < "2018-06-01")
]

In [ ]:
mars_weather_df.terrestrial_date.min(), mars_weather_df.terrestrial_date.max()

## Ensure data is ordered by time and equidistant with respect to time

In [ ]:
mars_weather_df.terrestrial_date = pd.to_datetime(mars_weather_df.terrestrial_date)
mars_weather_df.sort_values("terrestrial_date", inplace=True)
diff = mars_weather_df.terrestrial_date - mars_weather_df.terrestrial_date.shift(1)
diff.value_counts()

## Visualisations

In [ ]:
def plot_weather_df(weather_df): 
    time_earth = weather_df.terrestrial_date
    time_mars = weather_df.sol
    
    plt.figure(figsize=(20, 7))
    sns.lineplot(time_earth, weather_df.min_temp, label="min air temp")
    sns.lineplot(time_earth, weather_df.max_temp, label="max air temp")
    plt.ylabel("temperature")
    plt.show()
    
    plt.figure(figsize=(20, 7))
    sns.lineplot(time_earth, weather_df.min_gts_temp, label="max ground temp")
    sns.lineplot(time_earth, weather_df.max_gts_temp, label="min ground temp")
    plt.ylabel("temperature")
    plt.show()
    
    plt.figure(figsize=(20, 7))
    sns.lineplot(time_earth, weather_df.pressure)
    plt.show()

In [ ]:
plot_weather_df(mars_weather_df)

In [ ]:
mars_weather_df_sample = mars_weather_df[(mars_weather_df.terrestrial_date <= "2015-10-01" )]
plot_weather_df(mars_weather_df_sample)

## Downsample to more easily trends in the data

In [ ]:
weekly_resample_df = mars_weather_df.resample("7D", on="terrestrial_date").mean().reset_index(drop=False)
monthly_resample_df = mars_weather_df.resample("1M", on="terrestrial_date").mean().reset_index(drop=False)

In [ ]:
plot_weather_df(weekly_resample_df)

In [ ]:
plot_weather_df(monthly_resample_df)

## Time Series Decomposition: 
- Decompose into level, trend, seasonality and noise.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
def plot_signal_decomposition(df, column_name): 
    time_earth = df.terrestrial_date
    decomp = seasonal_decompose(df[column_name], period=52, model="additive", extrapolate_trend="freq")

    for quantity in [df[column_name], decomp.trend, decomp.seasonal, decomp.resid]: 
        plt.figure(figsize=(20, 7))
        sns.lineplot(time_earth, quantity)
        plt.show()

In [ ]:
mars_weather_df.terrestrial_date.min(), mars_weather_df.terrestrial_date.max()

In [ ]:
mars_weather_year= mars_weather_df[
    (mars_weather_df.terrestrial_date > "2014-01-01") & (mars_weather_df.terrestrial_date < "2017-01-01")
]
    

In [ ]:
plot_signal_decomposition(mars_weather_year, "max_temp")

In [ ]:
plot_signal_decomposition(mars_weather_df, "max_gts_temp")

In [ ]:
plot_signal_decomposition(mars_weather_df, "pressure")

## ACF and PACF
- Detect non-stationarity
- Determine the order of AR and MA terms in SARIMAX

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
def plot_acf_and_pacf(df, column_name): 
    f, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 8))

    plot_acf(df[column_name], lags=200, ax=ax[0])
    plot_pacf(df[column_name], lags=100, ax=ax[1])

    plt.show()


In [ ]:
plot_acf_and_pacf(mars_weather_df, "min_temp")

In [ ]:
plot_acf_and_pacf(mars_weather_df, "min_gts_temp")

In [ ]:
plot_acf_and_pacf(mars_weather_df, "pressure")

- Strong indication of non stationarity, since many lags before autocorrelation tends to 0

## Check for Stationarity

- Confirm non-stationarity with ADF test

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def apply_adf_test(df, column_name): 
    adf_test = adfuller(df[column_name].values)
    
    return adf_test

In [ ]:
for quantity in ["min_temp", "max_temp", "min_gts_temp", "max_gts_temp", "pressure"]:
    result = apply_adf_test(mars_weather_df, quantity)
    print(f"p-value for {quantity} is {result[1]}")

- Using 5% level of significance, out of all quantities only the p-value of the ADF test for pressure allows us to reject the null hypothesis i.e that the time series has a unit root and thus is non-stanionery. 
- So all temperature signals are non-stationery and must be corrected to account for this. 

## Determine order of differencing (integrated term)

Find the lowest order of differencing that gives:
- Time series with constant mean. 
- ACF that decays rapidly to 0 (means that there is no longer long term trends in the time series)

In [ ]:
mars_weather_df["min_temp_difference1"] = mars_weather_df.min_temp.diff()
mars_weather_df["max_temp_difference1"] = mars_weather_df.max_temp.diff()

In [ ]:
plot_acf_and_pacf(mars_weather_df.dropna(), "min_temp_difference1")

In [ ]:
plot_acf_and_pacf(mars_weather_df.dropna(), "max_temp_difference1")

In [ ]:
plt.figure(figsize=(20, 7))
sns.lineplot(mars_weather_df.terrestrial_date, mars_weather_df.min_temp)
sns.lineplot(mars_weather_df.terrestrial_date, mars_weather_df.min_temp_difference1)

In [ ]:
plt.figure(figsize=(20, 7))
sns.lineplot(mars_weather_df.terrestrial_date, mars_weather_df.max_temp)
sns.lineplot(mars_weather_df.terrestrial_date, mars_weather_df.max_temp_difference1)

- Only 1 order of differencing is required for time series with constant mean and ACF to deay quickly to 0. 

## Determine AR and MA terms
- From ACF and PACF plots of the differnced series above, both AR and MA terms should be set to 1. 
- Number of lags before falling to 0. 

## SARIMAX model

### Fit model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
mars_weather_df.reset_index(inplace=True)
mars_weather_df.head()

In [ ]:
train_size = int(len(mars_weather_df) * 0.9)
test_size = int(len(mars_weather_df)) - train_size

train_df = mars_weather_df.loc[:train_size]
test_df = mars_weather_df.loc[train_size:]

In [ ]:
#sarimax_model = SARIMAX(train_df.min_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 100))
sarimax_model = ARIMA(train_df.min_temp, order=(1, 1, 1), seasonal_order=(1, 1, 1, 350))

In [ ]:
model_results = sarimax_model.fit(method='innovations_mle', low_memory=True, cov_type='none')

In [ ]:
model_results.plot_diagnostics(figsize=(18, 8))


In [ ]:
preds = model_results.get_prediction(start=int(train_size*0.7))
pred_ci = preds.conf_int()
ax = train_df.min_temp.loc[int(train_size*0.7):].plot(label='Observed')
preds.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Day')
ax.set_ylabel('Min temp')
ax.set_title("In-sample predictions")
plt.legend()
plt.show()


In [ ]:
y_pred = preds.predicted_mean
y_train = train_df.loc[int(train_size*0.7):, "min_temp"].values
mse = ((y_pred - y_train)**2).mean()
rmse = np.sqrt(mse)

print("MSE: ", mse)
print("RMSE: ", rmse)

In [ ]:
print(mars_weather_df.shape)
print(train_size)
print(test_size)

In [ ]:
forecast_steps = 30

y_pred_forecast = model_results.get_forecast(steps=forecast_steps)
pred_ci = y_pred_forecast.conf_int()

ax = test_df.min_temp[:forecast_steps].plot(label='Observed', figsize=(14, 4))

y_pred_forecast.predicted_mean.plot(ax=ax, label='Forecast')

ax.fill_between(
    pred_ci.index,
    pred_ci.iloc[:, 0],
    pred_ci.iloc[:, 1], color='k', 
    alpha=.25
)

ax.set_xlabel('Day')
ax.set_ylabel('Min temp')
ax.set_title("Out-of-sample predictions")
plt.legend()
plt.show()